# Deliverable 2
This is the notebook that has the second deliverable.

# Data reading
Following code block shows we can read in the data, and that's pretty much it.

In [ ]:
import data_readers

# The links and edges
wikispeedia = data_readers.read_wikispeedia_graph()

# The finished paths
finished_paths = data_readers.read_finished_paths()

# The unfinished paths
unfinished_paths = data_readers.read_unfinished_paths()

# The shortest path matrix
# This one is the slowest to read by far, probably due to the weird parsing that has to be done!
shortest_path_df = data_readers.read_shortest_path_df()

# Searching for the string of a given article. It has to be formatted like the article name
# Which shouldn't be a problem, as we'll probably usually retrieve them internally
obi_wan_text = data_readers.plaintext_article_finder('Obi-Wan_Kenobi')